In [ ]:
########### config ############
delta_lake_path = '../CricketDataService_New/deltalake/main/players' #local
# delta_lake_path = 'az://sports/final_players_data' #azure
options={"AZURE_STORAGE_ACCOUNT_NAME":"jioaishared",
 "AZURE_STORAGE_ACCOUNT_KEY":"OYRIjKqneeiI+UrDyLHV7CBOudfdxb73AqkdtUYX3UGYVynUDjQdCtv60a73oEf++qIYnY6844QUOSPrcndLuQ=="}
init_data_path = '../Data/Squad/init/'
test_data_path = '../Data/Squad/test/'
main_data_path = '../Data/Squad/main/'

In [ ]:
############ packages ###########
import pandas as pd
import time
import json
import os
from pathlib import Path
from datetime import datetime, timedelta
import pyarrow as pa 
from deltalake import DeltaTable
from deltalake.writer import write_deltalake
from azure.storage.filedatalake import DataLakeFileClient
from azure.storage.blob import BlobServiceClient, generate_account_sas, ResourceTypes, AccountSasPermissions

In [ ]:
############ utils ###########

def convert_dataframe_to_json_list(dataframe):
    data_dict = dataframe.to_dict(orient='records')
    json_list = [json.dumps(data) for data in data_dict]
    return json_list
def convert_id_to_integer(id_string, range_size):
    id_hash = hash(id_string)  
    id_integer = abs(id_hash) % range_size  
    return id_integer
def read_and_parse_file(file_path):
    with open(file_path, "r") as file:
        content = file.read()
    content = content.strip()[8:-2]
    content = json.loads(content)
    return content
def read_json_files(folder_path):
    data_list = []
    for path in Path(folder_path).rglob("*.js"):
        content = read_and_parse_file(path)
        data_list.append(content)
    return data_list
def convert_data_to_dataframe(data):
    dataframe_list = []
    for item in data:
        df = pd.DataFrame({
            "player_id": item["PlayerID"],
            "team_id": item["TeamID"],
            "team_image": item['TeamImage'],
            "player_image": item['PlayerImage'],
            "team_code": item['TeamCode'],
            "team_name": item["TeamName"],
            "player_name": item["PlayerName"],
            "player_short_name": item["PlayerShortName"],
            "player_skill": item["PlayerSkill"],
            "batting_type": item["BattingType"],
            "bowling_proficiency": item["BowlingProficiency"],
            "is_captain": item["IsCaptain"],
            "is_vice_captain": item["IsViceCaptain"]
        }, index=range(1))
        dataframe_list.append(df)
    return pd.concat(dataframe_list)
def convert_data_list(data_list):
    dataframe_list = []
    for json_data in data_list:
        squad_a_dataframe = convert_data_to_dataframe(json_data["squadA"])
        squad_b_dataframe = convert_data_to_dataframe(json_data["squadB"])
        dataframe_list.append(pd.concat([squad_a_dataframe, squad_b_dataframe]))
    return pd.concat(dataframe_list)
def convert_null_values(dataframe):
    # Check for null values in the dataframe
    null_values = dataframe.isnull()
    
    # Iterate over each column
    for column in dataframe.columns:
        # Check if the column contains null values
        if null_values[column].any():
            # Check if the column is of integer type
            if dataframe[column].dtype == 'int64':
                # Convert integer null values to -1
                dataframe[column] = dataframe[column].fillna(-1)
            else:
                # Convert character null values to "NA"
                dataframe[column] = dataframe[column].fillna("NA")
    
    return dataframe
def convert_data_to_df(data_list):
    dataframe_list = []
    for json_data in data_list:
        squad_a_dataframe = convert_data_to_dataframe(json_data["squadA"])
        squad_b_dataframe = convert_data_to_dataframe(json_data["squadB"])
        dataframe_list.append(pd.concat([squad_a_dataframe, squad_b_dataframe]))
    return dataframe_list
def update_players_data(new_players_df_list, existing_players_df):    
    # Measure execution time for the entire function
    start_time = time.time()

    for i, new_players in enumerate(new_players_df_list):
        if i != 0:
            existing_players_df = updated_players
        
        new_players.set_index('player_id', inplace=True)

        # Measure execution time for updating existing_players_df with new_players df
        update_start_time = time.time()

        existing_players_ids = set(existing_players_df.index)
        new_players_ids = set(new_players.index)
        common_ids = existing_players_ids.intersection(new_players_ids)

        existing_players_common_mask = existing_players_df.index.isin(common_ids)
        new_players_common_mask = new_players.index.isin(common_ids)

        update_columns = ['team_id', 'team_name', 'team_code', 'team_image', 'is_captain', 'is_vice_captain']

        existing_players_update_dataframe = existing_players_df.loc[existing_players_common_mask, update_columns]
        new_players_update_dataframe = new_players.loc[new_players_common_mask, update_columns]

        existing_players_update_dataframe.update(new_players_update_dataframe)

        updated_players = pd.concat([existing_players_df, new_players.loc[~new_players_common_mask]])

        # Measure execution time for updating existing_players_df
        update_end_time = time.time()
        update_time = update_end_time - update_start_time
        # print(f"Update time for iteration {i}: {update_time} seconds")

    # Measure execution time for the entire function
    end_time = time.time()
    total_time = end_time - start_time
    # print(f"Total execution time: {total_time} seconds")

    return updated_players


In [ ]:
############ load initial data 
init_players = convert_data_list(read_json_files(init_data_path))

########### create new delta table in delta lake and add initial data into that 
write_deltalake(delta_lake_path,init_players,overwrite_schema=True,mode='overwrite',storage_options=options) ## neither overwrite or append mode 



In [ ]:
############## read existing player's data as delta table
existing_players_delta_table = DeltaTable(delta_lake_path,storage_options=options)

############## get delta table schema 
existing_players_delta_table_schema = existing_players_delta_table.schema()

############ convert delta table to dataframe and set index
existing_players_df = existing_players_delta_table.to_pandas()
existing_players_df.set_index('player_id',inplace=True)

print(f'{len(existing_players_df)} players are already in dl')


In [ ]:
new_players_df_list = convert_data_to_df(read_json_files(main_data_path))
print(f'players from {len(new_players_df_list)} matches need to get updated in dl')

In [ ]:
updated_players_df = update_players_data(new_players_df_list,existing_players_df)
print(f'now there are {len(updated_players_df)} players')

In [ ]:
########### write updated player's data to delta lake ############

write_deltalake(delta_lake_path,updated_players,mode='overwrite',storage_options=options,overwrite_schema=True)
